In [1]:
#include <stdio.h>
#include <string>

In [2]:
#pragma cling add_include_path("/home/nimalanm/Documents/opt/include")
#pragma cling load("cfitsio")
#pragma cling load("wcs")

In [25]:
#include "fitsio.h"
#include "wcslib/wcs.h"
#include "wcslib/wcshdr.h"

In [26]:
fitsfile *fptr;
int status = 0;

In [27]:
std::string file = "image.i.08161181-7039447.taylor.0.restored.fits";

In [29]:
#include <vector>

int numAxes = 0;
char* hdr;
int nkeys;
int nexc;

try {
    
    if (!fits_open_image(&fptr, file.c_str(), READONLY, &status)) {

        if (fits_get_img_dim(fptr, &numAxes, &status)) {
            printf("Error reading dimensions \n");
            return 1;
        }
        
        std::vector<size_t> dimAxes(numAxes, 1);
        
        if (fits_get_img_size(fptr, numAxes, (long*)dimAxes.data(), &status)) {
            printf("Unable to read axis size\n");
            return 1;
        }
        
        // Print our image dimensions
        for (int i=0; i<numAxes; i++)
            printf("%lu ", dimAxes[i]);
        printf("\n");
        
        // Read the fits headers
        fits_hdr2str(fptr, 1, NULL, nexc, &hdr, &nkeys, &status);
        if (status) {
            printf("Error reading header");
            return 1;
        }
        
        printf("Header\n");
        printf("%s\n", hdr);
        
        struct wcsprm *localwcs = (struct wcsprm*)calloc(1, sizeof(struct wcsprm));
        localwcs->flag=-1;
        int localnwcs;
        
        if (wcsini(true, numAxes, localwcs)) {
            printf("Error creating WCS\n");
            return 1;
        }
        
        int relax = 1;
        int ctrl = 2;
        int nreject;
        
        // Parse WCS from FITS Header
        if (wcspih(hdr, nkeys, relax, ctrl, &nreject, &localnwcs, &localwcs)) {
            printf("Error parsing WCS Header");
        }
        
        int stat[NWCSFIX];
        if (wcsfix(1, (const int*)dimAxes, localwcs, stat)) {
            printf("Error fixing WCS\n");
            return 1;
        }
        
        if (wcsset(localwcs)) {
            printf("Error creating wcsstruct");
            return 1;
        }
        
        // Cleanup
        if (localwcs)  {
            wcsvfree(&localnwcs, &localwcs);
            wcsfree(localwcs);
            free(localwcs);
        }
        if (hdr) {
            free(hdr);
        }
        fits_close_file(fptr, &status);
    } else {
        printf("Unable to open fits \n");
        throw 1;
    }
    
} catch(int status) {
    printf("Execution failed %d \n", status);
}


2048 2048 1 1 
Header
SIMPLE  =                    T /Standard FITS                                   BITPIX  =                  -32 /Floating point (32 bit)                         NAXIS   =                    4                                                  NAXIS1  =                 2048                                                  NAXIS2  =                 2048                                                  NAXIS3  =                    1                                                  NAXIS4  =                    1                                                  BSCALE  =   1.000000000000E+00 /PHYSICAL = PIXEL*BSCALE + BZERO                 BZERO   =   0.000000000000E+00                                                  BUNIT   = 'Jy/beam '           / Brightness (pixel) unit                        EQUINOX =   2.000000000000E+03                                                  RADESYS = 'FK5     '                                                            LONPOLE =   1.8000